## Introduction



This tutorial is an introduction to the syntax used by the optimizer, as well as the principles of Bayesian optimization in general.

We'll start by minimizing Himmelblau's function, which looks like this:

In [ ]:
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt

import bloptools

x1 = x2 = np.linspace(-5.0, 5.0, 256)
X1, X2 = np.meshgrid(x1, x2)

plt.pcolormesh(x1, x2, bloptools.experiments.tests.himmelblau(X1, X2), norm=mpl.colors.LogNorm(), shading="auto")

plt.xlabel("x1")
plt.ylabel("x2")
plt.colorbar()

In [ ]:
from bloptools.objects import TimeReadback

tr = TimeReadback(name="timestamp")

tr.read()

There are several things that our agent will need. The first ingredient is some degrees of freedom (these are always `ophyd` devices) which the agent will move around to different inputs within each DOF's bounds (the second ingredient). We define these here:

In [ ]:
import bloptools

dofs = bloptools.objects.get_dummy_dofs(n=2)  # get a list of two DOFs
bounds = [(-5.0, +5.0), (-5.0, +5.0)]

The agent automatically samples at different inputs, but we often need some post-processing after data collection. In this case, we need to give the agent a way to compute Himmelblau's function. We accomplish this with a digestion function:

In [ ]:
import bloptools

dofs = bloptools.objects.get_dummy_dofs(n=2)  # get a list of two DOFs
bounds = [(-5.0, +5.0), (-5.0, +5.0)]

from bloptools.objects import TimeReadback

tr = TimeReadback(name="timestamp")

tr.read()


def digestion(db, uid):
    table = db[uid].table()
    products = pd.DataFrame()

    for index, entry in table.iterrows():
        products.loc[index, "himmelblau"] = bloptools.test_functions.himmelblau(entry.x1, entry.x2)

    return products


from bloptools.tasks import Task

task = Task(key="himmelblau", kind="min")

The next ingredient is a task, which gives the agent something to do. We want it to minimize Himmelblau's function, so we make a task that will try to minimize the output of the digestion function called "himmelblau". We also include a transform function, which will make it easier to regress over the outputs of the function.

In [ ]:
from bloptools.tasks import Task

task = Task(key="himmelblau", kind="min")

Combining all of these with a databroker instance, we can make an agent:

In [ ]:
%run -i ../../../examples/prepare_bluesky.py # prepare the bluesky environment

boa = bloptools.bayesian.Agent(
    dofs=dofs,
    bounds=bounds,
    passive_dims=[tr],
    tasks=task,
    digestion=digestion,
    db=db,
)

RE(boa.initialize(init_scheme="quasi-random", n_init=32))

We initialized the GP with the "quasi-random" strategy, as it doesn't require any prior data. We can view the state of the optimizer's posterior of the tasks over the input parameters:

In [ ]:
np.atleast_1d([]).size

In [ ]:
boa.plot_tasks()

We want to learn a bit more, so we can ask the agent where to sample based off of some strategy. Here we use the "esti" strategy, which maximizes the expected sum of tasks improvement.

In [ ]:
RE(boa.learn(strategy="esti", n_iter=4))
boa.plot_tasks()

The agent has updated its model of the tasks, including refitting the hyperparameters. Continuing:

In [ ]:
RE(boa.learn(strategy="esti", n_iter=16))
boa.plot_tasks()

Eventually, we reach a point of saturation where no more improvement takes place:

In [ ]:
RE(boa.learn(strategy="esti", n_iter=32))
boa.plot_tasks()

In [ ]:
boa.tasks[0].regressor.covar_module.base_kernel.trans_matrix